In [ ]:
# Import Packages 

In [1]:
import re
from copy import copy

import pandas as pd
import numpy as np
from shapely.geometry import Point
import geopandas as gpd
from sklearn.neighbors import BallTree

In [209]:
def average_str(s):
    s = s.split(" (")[0].split("-")
    s = [float(s) for s in s]
    s = np.average(s)
    
def add_sg_data_to_tank_data(tank_data, chemical_data):
    sg_data = []
    for cas_number in tank_data["cas_number"]:
        if cas_number is None:
            sg_data.append(None)
        elif cas_number is np.NaN:
            sg_data.append(None)
        else:
            subset_of_chemical_data_by_tank = chemical_data[chemical_data['CAS#'].isin(cas_number)]
            if len(subset_of_chemical_data_by_tank) > 0:
                sg_data.append(subset_of_chemical_data_by_tank)
            else:
                sg_data.append(None)
    tank_data["sg"] = sg
    return tank_data

In [2]:
tile_level_annotations = gpd.read_file("/hpc/group/borsuklab/ast/tile_level_annotation_multiple_capture_date_neighbor_tile_removed/tile_level_annotation_multiple_capture_date_neighbor_tile_removed.geojson")

In [3]:
# Read in tri data #/work/csr33/spatial_matching/tri/
tri_2022_us_path = "2022_us.csv"
tri_2022_us = pd.read_csv(tri_2022_us_path)

/tmp/ipykernel_1750990/113328475.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  tri_2022_us = pd.read_csv(tri_2022_us_path)


In [4]:
#add geometries
Geometry = [Point(xy) for xy in zip(tri_2022_us['13. LONGITUDE'], tri_2022_us['12. LATITUDE'])] 
tri_2022_us = gpd.GeoDataFrame(tri_2022_us, crs="EPSG:4326", geometry=Geometry)

#remove rows without locations
tri_2022_us = tri_2022_us[~np.isnan(tri_2022_us["12. LATITUDE"])]
tri_2022_us = tri_2022_us[~np.isnan(tri_2022_us["13. LONGITUDE"])]

#subset tri data based on naics codes
naics_industry_codes = pd.read_csv('/hpc/home/csr33/spatial-match-ast-chemicals/naics_industry_keys.csv')
tri_2022_us =tri_2022_us[tri_2022_us["19. INDUSTRY SECTOR CODE"].isin(naics_industry_codes["2022 NAICS US Code"].tolist())]

# Get unique tri locations
unique_tri_2022_us_values, unique_tri_2022_us_indices =  np.unique(tri_2022_us[["12. LATITUDE","13. LONGITUDE"]].values, return_index= True, axis = 0)
tri_2022_us_unique_locations = tri_2022_us.iloc[unique_tri_2022_us_indices]

# Create a BallTree for quick nearest neighbor lookup
btree = BallTree(tri_2022_us_unique_locations.geometry.apply(lambda x: (x.x, x.y)).tolist()) 

# Find closest point for each polygon
closest_points = []
for polygon in tile_level_annotations.geometry:
    point = polygon.representative_point()
    # Query ball tree to find closest point
    dist, idx = btree.query([(point.x, point.y)], k=1) 
    if dist > 0.1: 
        # No point within 1 km
        closest_points.append(None)
    else:
        closest_points.append(tri_2022_us_unique_locations.geometry.iloc[idx[0][0]])
        
# Add closest points back to polygons GeoDataFrame
tile_level_annotations['closest_point'] = closest_points

# Group by 'geometry' and aggregate 'name' and 'value' into lists
tri_2022_us_chemical = tri_2022_us.groupby('geometry').agg({"34. CHEMICAL": list, "37. CAS#": list}).reset_index()

#add chemical data to tile level annotations
merged_df = pd.merge(tile_level_annotations, tri_2022_us_chemical, left_on='closest_point', right_on='geometry', how='left')
tile_level_annotations[["chemical_name", "cas_number"]] = merged_df[["34. CHEMICAL", "37. CAS#"]]

In [215]:
len([1,2,2,3,4,5,6])//2

3

# read in and formal chemical data

In [183]:
chemical_data = pd.read_csv("/hpc/group/borsuklab/csr33/chemical_data/niosh_pocket_guide/NIOSH Pocket Guide.csv")
# remove rows where specific gravity is none
chemical_data = chemical_data.dropna(subset=['Specific gravity'])
# remove rows where specific gravity is for a metal
# Remove rows with 'metal'
chemical_data = chemical_data[~chemical_data['Specific gravity'].str.contains('metal', case=False)]
chemical_data = chemical_data[~chemical_data['Specific gravity'].isin(["?", "? "])]
chemical_data['Specific gravity'] = [str(average_str(sg)) if "-" in sg else sg for sg in chemical_data['Specific gravity']]
chemical_data['Specific gravity'] = [float(re.search(r'(\d+(\.\d+)?)', sg).group(0)) for sg in chemical_data['Specific gravity']]

In [203]:
tile_level_annotations = add_sg_data_to_tank_data(tile_level_annotations, chemical_data)

In [211]:
import os
ast_scaled_data = pd.read_parquet(os.path.join("/work/csr33/fragility", "model_data", "buckling", 
                                      f'buckling_scaled_simulated_data.parquet'), engine='pyarrow')

In [212]:
ast_scaled_data

diameter  external_tank_height  \
id     category  __null_dask_index__                                   
61768  Category5 38284000                  4.8              6.367624   
                 38284001                  4.8              7.224267   
                 38284002                  4.8              8.554616   
                 38284003                  4.8              8.631876   
                 38284004                  4.8              5.835329   
...                                        ...                   ...   
127438 Category4 34455595                  7.8             16.724643   
                 34455596                  7.8             17.993693   
                 34455597                  7.8              7.833560   
                 34455598                  7.8             14.493391   
                 34455599                  7.8              9.596480   

                                      tank_shell_thickness  \
id     category  __null_dask_index__                         
61768  Category5 38284000                            0.005   
                 38284001                            0.005   
                 38284002                            0.005   
                 38284003                            0.005   
                 38284004                            0.005   
...                                                    ...   
127438 Category4 34455595                            0.005   
                 34455596                            0.005   
                 34455597                            0.005   
                 34455598                            0.005   
                 34455599                            0.005   

                                      tank_contents_specific_gravity  \
id     category  __null_dask_index__                                   
61768  Category5 38284000                                   0.800623   
                 38284001                                   0.798770   
                 38284002                                   0.825083   
                 38284003                                   0.794375   
                 38284004                                   0.807424   
...                                                              ...   
127438 Category4 34455595                                   0.917315   
                 34455596                                   0.908443   
                 34455597                                   0.818688   
                 34455598                                   0.814214   
                 34455599                                   0.803713   

                                      inundation  velocity  flotation_Y  \
id     category  __null_dask_index__                                      
61768  Category5 38284000                    8.5  4.189224          1.0   
                 38284001                    8.5  5.727290          1.0   
                 38284002                    8.5  5.415298          1.0   
                 38284003                    8.5  4.833314          1.0   
                 38284004                    8.5  4.681660          1.0   
...                                          ...       ...          ...   
127438 Category4 34455595                   13.5  5.925903          1.0   
                 34455596                   13.5  7.843119          1.0   
                 34455597                   13.5  7.123234          1.0   
                 34455598                   13.5  5.108177          1.0   
                 34455599                   13.5  5.676439          1.0   

                                      sliding_Y  flood_shell_buckling_Y  \
id     category  __null_dask_index__                                      
61768  Category5 38284000                   0.0                     1.0   
                 38284001                   0.0                     1.0   
                 38284002                   0.0                     1.0   
      